This notebook compares the scores of the local models and the model chain

In [1]:
import numpy as np
import pandas as pd
#import itertools
#import random

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, accuracy_score, mean_squared_error as mse, brier_score_loss
from chaining import Chain
import os

In [2]:
def missingness_stratified_cv(df, N_FOLDS=5, random_state=None):
    # Add seed for reproducibility of the predictions (to get the same scores each time we run the code)
    np.random.seed(random_state)

    # Initial complete-case test fold assignment
    cv = pd.Series(np.nan, index=df.index)
    i_cc = (df.isna().sum(axis=1) == 0) # Complete cases
    cv.iloc[i_cc] = np.random.randint(low=0, high=N_FOLDS, size=i_cc.sum())

    # Go over columns from most missing to least missing
    for j in df.isna().sum().argsort()[::-1]:
        # Instances i that are not assigned yet but for which df[i,j] is observed
        i_tbf = (cv.isna()) & (~df.iloc[:,j].isna()) # to be filled
        # Fill them randomly
        cv.iloc[i_tbf] = np.random.randint(low=0, high=N_FOLDS, size=i_tbf.sum())

    return cv

In [3]:
possible_paths = [
    'C:/Users/lenne/OneDrive/Documenten/Master of Statistics and Data Science/2023-2024/Master thesis/Thesis_Sofia_Lennert/new_data',
    'C:/Users/anaso/Desktop/SOFIA MENDES/KU Leuven/Master Thesis/Thesis_Sofia_Lennert/new_data'
]

# Define file names
file = 'merged_data.csv'

# Find full paths to the CSV files
path = next((f'{path}/{file}' for path in possible_paths if os.path.exists(f'{path}/{file}')), None)

data = pd.read_csv(path)

# Bin the number of relapses into 0, 1, 2, 3 and 4+ 
def bin_column(value):
    if value in [0, 1, 2, 3]:
        return str(value)
    else:
        return '4+'
data['NRELAP'] = data['NRELAP'].apply(bin_column)

# Resulting DataFrame will have aggregated data from all four datasets based on the specific_column
pd.set_option('display.max_columns', None)
data

,USUBJID,AGE,SEX,RACE,CONTINENT,CESEV,CECONTRT,TOTRELAP,MHCONTRT,MHDIAGN,CARDIO,URINARY,MUSCKELET,FATIGUE,SMSTDY,NRELAP,NHPT-before,NHPT-2y,NHPT-after_2y,PASAT_2s-before,PASAT_2s-2y,PASAT_2s-after_2y,PASAT_3s-before,PASAT_3s-2y,PASAT_3s-after_2y,SDMT-before,SDMT-2y,T25FW-before,T25FW-2y,T25FW-after_2y,T-before,T-after,P-before,P-after,N-before,N-after,SLEC_before,SLEC_after,SES_after,SES_before,VAA,BDI-before,BDI-after,EDSS-before,EDSS-2y,EDSS-after_2y,KFSS1-Sensory-2y,KFSS1-Sensory-after_2y,KFSS1-Sensory-before,KFSS1-Brain-2y,KFSS1-Brain-after_2y,KFSS1-Brain-before,KFSS1-Bowel-2y,KFSS1-Bowel-after_2y,KFSS1-Bowel-before,KFSS1-Pyramidal-2y,KFSS1-Pyramidal-after_2y,KFSS1-Pyramidal-before,KFSS1-Cerebral-2y,KFSS1-Cerebral-after_2y,KFSS1-Cerebral-before,KFSS1-Visual-2y,KFSS1-Visual-after_2y,KFSS1-Visual-before,KFSS1-Cerebellar-2y,KFSS1-Cerebellar-after_2y,KFSS1-Cerebellar-before,KFSS_M-2y,KFSS_M-after_2y,KFSS_M-before,KFSS_P-2y,KFSS_P-after_2y,KFSS_P-before,M_R36-SF12-before,P_R36-SF12-before,R36-SF12-before_Ind,M_R36-SF12-after,P_R36-SF12-after,R36-SF12-after_Ind
0,MSOAC/0014,46.0,F,NaN,NaN,NaN,NaN,NaN,NaN,RRMS,0,0,0,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MSOAC/0016,NaN,M,WHITE,NORTH AMERICA,NaN,NaN,NaN,Y,SPMS,1,1,0,1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.55,6.60,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MSOAC/0019,44.0,M,NON-WHITE,NaN,NaN,NaN,NaN,NaN,PPMS,1,1,0,0,NaN,0.0,23.65,21.30,20.15,34.5,35.5,43.0,43.5,51.0,53.0,NaN,NaN,6.30,6.15,5.85,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.75,3.50,3.0,0.333333,0.166667,0.500000,0.2,0.0,0.2,0.000000,0.166667,0.083333,0.333333,0.5,0.416667,0.0,0.0,0.0,0.333333,0.0,0.333333,0.0,0.2,0.0,0.185185,0.185185,0.240741,0.166667,0.083333,0.208333,0.828571,0.772152,1.0,0.857143,0.721519,1.0
3,MSOAC/0024,60.0,M,WHITE,NORTH AMERICA,NaN,NaN,NaN,NaN,SPMS,1,1,1,1,NaN,0.0,34.45,37.50,NaN,55.0,54.0,NaN,60.0,60.0,NaN,NaN,NaN,4.50,5.25,NaN,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,0.031746,0.023810,4.00,3.75,NaN,0.333333,NaN,0.333333,0.0,NaN,0.1,0.583333,NaN,0.666667,0.166667,NaN,0.250000,0.0,NaN,0.0,0.000000,NaN,0.083333,0.2,NaN,0.5,0.129630,NaN,0.240741,0.291667,NaN,0.375000,0.885714,0.569620,1.0,0.857143,0.716216,1.0
4,MSOAC/0030,28.0,F,WHITE,EUROPE,NaN,NaN,NaN,NaN,RRMS,1,1,0,1,NaN,0.0,16.55,17.90,NaN,NaN,NaN,NaN,58.0,60.0,NaN,63.5,69.0,4.85,4.70,NaN,0.0,0.0,0.0,0.0,0.0,0.0,26.0,24.0,1.25,1.25,NaN,0.063492,0.039683,2.00,1.50,NaN,0.166667,NaN,0.166667,0.2,NaN,0.2,0.166667,NaN,0.166667,0.166667,NaN,0.333333,0.0,NaN,0.2,0.166667,NaN,0.083333,0.0,NaN,0.1,0.111111,NaN,0.203704,0.166667,NaN,0.125000,0.933333,0.846154,0.0,0.833333,0.730769,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2460,MSOAC/9986,46.0,M,WHITE,OCEANIA,NaN,NaN,NaN,NaN,RRMS,1,1,0,1,NaN,0.0,19.35,18.95,NaN,NaN,NaN,NaN,58.0,60.0,NaN,51.0,60.0,3.90,3.80,NaN,0.0,0.0,0.0,0.0,0.0,0.0,36.0,35.0,1.25,1.25,NaN,0.047619,0.063492,2.75,2.50,NaN,0.333333,NaN,0.166667,0.0,NaN,0.0,0.333333,NaN,0.250000,0.166667,NaN,0.333333,0.0,NaN,0.0,0.000000,NaN,0.000000,0.0,NaN,0.2,0.111111,NaN,0.148148,0.166667,NaN,0.125000,0.833333,0.730769,0.0,0.800000,0.750000,0.0
2461,MSOAC/9987,18.0,F,NaN,NaN,NaN,NaN,NaN,NaN,RRMS,0,0,0,0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [4]:
#variables = ['KFSS_M-2y', 'EDSS-2y', 'T25FW-2y', 'NRELAP']# removed KFSS_P-2y for now -- ('SMSTDY' gave a score of -0.03)
variables = ['KFSS_M-2y', 'KFSS_P-2y', 'EDSS-2y', 'T25FW-2y', 'NHPT-2y', 'P_R36-SF12-after', 'M_R36-SF12-after', 
             'SES_after', 'SLEC_after', 'KFSS_M-after_2y', 'KFSS_P-after_2y', 'EDSS-after_2y', 'NRELAP', 'CESEV']

In [5]:
# Extract targets
targets = data[variables]

# Extract features by dropping the target columns
#features = data.drop(variables, axis=1)

columns_to_keep = ['AGE', 'SEX', 'RACE', 'CONTINENT', 'MHDIAGN', 'CARDIO', 'URINARY', 'MUSCKELET', 'FATIGUE']
                    #'PASAT_2s-before', 'PASAT_3s-before', 'SDMT-before', 'BDI-before', 'T-before','P-before','N-before']
# removed: 'NHPT-before', 'T25FW-before', 'SLEC_before','SES_before','EDSS-before', 'KFSS_M-before', 'KFSS_P-before', 'M_R36-SF12-before', 
# 'P_R36-SF12-before', 'R36-SF12-before_Ind',

features = data[columns_to_keep]
features

,AGE,SEX,RACE,CONTINENT,MHDIAGN,CARDIO,URINARY,MUSCKELET,FATIGUE
0,46.0,F,NaN,NaN,RRMS,0,0,0,0
1,NaN,M,WHITE,NORTH AMERICA,SPMS,1,1,0,1
2,44.0,M,NON-WHITE,NaN,PPMS,1,1,0,0
3,60.0,M,WHITE,NORTH AMERICA,SPMS,1,1,1,1
4,28.0,F,WHITE,EUROPE,RRMS,1,1,0,1
...,...,...,...,...,...,...,...,...,...
2460,46.0,M,WHITE,OCEANIA,RRMS,1,1,0,1
2461,18.0,F,NaN,NaN,RRMS,0,0,0,0
2462,38.0,F,NaN,NaN,RRMS,0,0,0,0
2463,40.0,F,WHITE,NaN,PPMS,0,1,0,1


In [6]:
object_columns = features.select_dtypes(include=['object'])
features = pd.get_dummies(features, columns=object_columns.columns, dtype=int)
features.head()

,AGE,CARDIO,URINARY,MUSCKELET,FATIGUE,SEX_F,SEX_M,RACE_NON-WHITE,RACE_WHITE,CONTINENT_ASIA,CONTINENT_EURASIA,CONTINENT_EUROPE,CONTINENT_NORTH AMERICA,CONTINENT_OCEANIA,CONTINENT_SOUTH AMERICA,MHDIAGN_PPMS,MHDIAGN_RRMS,MHDIAGN_SPMS
0,46.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
1,NaN,1,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1
2,44.0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0
3,60.0,1,1,1,1,0,1,0,1,0,0,0,1,0,0,0,0,1
4,28.0,1,1,0,1,1,0,0,1,0,0,1,0,0,0,0,1,0


In [7]:
targets.dtypes

KFSS_M-2y           float64
KFSS_P-2y           float64
EDSS-2y             float64
T25FW-2y            float64
NHPT-2y             float64
P_R36-SF12-after    float64
M_R36-SF12-after    float64
SES_after           float64
SLEC_after          float64
KFSS_M-after_2y     float64
KFSS_P-after_2y     float64
EDSS-after_2y       float64
NRELAP               object
CESEV                object
dtype: object

In [8]:
# Set random state for reproducibility
random_state = 42
N_FOLDS = 5

In [9]:
# Generate CV folds
cv=missingness_stratified_cv(features, N_FOLDS, random_state)
cv = cv.to_frame(name="CV Fold")

features_cv = pd.merge(features, pd.DataFrame(cv), left_index=True, right_index=True)
targets_cv = pd.merge(targets, pd.DataFrame(cv), left_index=True, right_index=True)

features_cv['CV Fold'].value_counts()

CV Fold
4.0    510
3.0    502
0.0    500
1.0    495
2.0    458
Name: count, dtype: int64

Is it a problem that not all folds have the exact same number?

In [10]:
def normalized_mean_squared_error(true, pred):
    num = mse(true, pred)
    mean_value = np.mean(true)
    mean = np.full_like(true, mean_value)
    den = mse(true, mean)
    nmse_loss = num/den
    #rrmse_loss = np.sqrt(squared_error)
    return nmse_loss

---

### Local Models

In [11]:
y_pred_list = []
y_test_list = []
y_pred_prob_list = []
yi_test_dummies_list = []

for i in range(0, N_FOLDS): 
    Xi_train = features_cv[features_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = features_cv[features_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))

    # One hot encode categorical targets of test set to be able to compute brier score
    subset_yi_test = yi_test.select_dtypes(include=['object'])
    yi_test_dummies = pd.get_dummies(subset_yi_test, columns=subset_yi_test.columns, dtype=int)
    

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate=False, #RUN LOCAL MODELS 
    )
    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_prob = chain.predict_proba(Xi_test)
    y_pred_list.append(y_pred)
    y_pred_prob_list.append(y_pred_prob)
    yi_test_dummies_list.append(yi_test_dummies)
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [12]:
yi_test_dummies_avg = []
# Calculate the percentage of 1s in each column
for yi_test_dummies_fold in yi_test_dummies_list:

    percentages = yi_test_dummies_fold.sum() / len(yi_test_dummies_fold)

    yi_test_dummies_avg_fold = pd.DataFrame(0, index=yi_test_dummies_fold.index, columns=yi_test_dummies_fold.columns)

    # Replace values in each column with the corresponding percentage
    for col in yi_test_dummies_avg_fold.columns:
        yi_test_dummies_avg_fold[col] = yi_test_dummies_fold[col].apply(lambda x: percentages[col])

    yi_test_dummies_avg.append(yi_test_dummies_avg_fold)

In [13]:
# Initialize an empty list to store the concatenated DataFrames
concatenated_dfs = []

# Iterate over each pair of arrays
for j, fold in enumerate(y_pred_prob_list):
    dfs = []
    len_array = 0
    
    for i, array in enumerate(fold):
        # Convert array to DataFrame
        col = yi_test_dummies_list[j].columns[len_array:len_array+len(array[0])]
        df = pd.DataFrame(array, columns=col, index=yi_test_dummies_list[j].index)
        dfs.append(df)
        len_array += len(array[0])
    
    # Concatenate DataFrames
    concatenated_df = pd.concat(dfs, axis=1)
    concatenated_dfs.append(concatenated_df)

# Now you should have a list of concatenated DataFrames

In [14]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Iterate over each outcome variable in the folds
for variable_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][variable_name] 
        y_prob = concatenated_dfs[fold_index][variable_name] 
        y_prob_avg = yi_test_dummies_avg[fold_index][variable_name] 
        
        # Check if the target variable is numerical or categorical
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)

        normalized_brier= brier_score/brier_baseline
                  
        variable_scores.append(normalized_brier)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each level:")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")

Normalized Brier scores for each level:
NRELAP_0.0: 1.02 (± 0.06)
NRELAP_1.0: 1.13 (± 0.04)
NRELAP_2.0: 1.18 (± 0.08)
NRELAP_3.0: 1.20 (± 0.10)
NRELAP_4+: 1.15 (± 0.07)
CESEV_MILD: 1.63 (± 0.15)
CESEV_MODERATE: 2.26 (± 0.16)
CESEV_SEVERE: 1.55 (± 0.10)


In [15]:
# Initialize dictionary to store column sums
column_sums = {}

# Iterate over each dataframe in reorganized_dummy_list_first
for df in yi_test_dummies_list:
    # Iterate over each column in the dataframe
    for column in df.columns:
        # Sum occurrences of 1s in the column and update column_sums
        column_sum = df[column].sum()
        column_sums[column] = column_sums.get(column, 0) + column_sum

# Create a dataframe from the column sums
total_counts_df = pd.DataFrame(list(column_sums.items()), columns=['Name', 'Total'])
total_counts_df

,Name,Total
0,NRELAP_0.0,1612
1,NRELAP_1.0,485
2,NRELAP_2.0,211
3,NRELAP_3.0,82
4,NRELAP_4+,75
5,CESEV_MILD,238
6,CESEV_MODERATE,534
7,CESEV_SEVERE,168


In [16]:
# Calculate the weighted sum for NRELAP
nrelap_weighted_sum = 0
total_count = 0
for level_name, avg_score, _ in scores_with_std:
    if level_name.startswith("NRELAP"):
        count = total_counts_df.loc[total_counts_df['Name'] == level_name, 'Total'].values[0]
        nrelap_weighted_sum += avg_score * count
        total_count += count
nrelap_weighted_avg = nrelap_weighted_sum / total_count

# Calculate the weighted sum for CESEV
cesev_weighted_sum = 0
total_count = 0
for level_name, avg_score, _ in scores_with_std:
    if level_name.startswith("CESEV"):
        count = total_counts_df.loc[total_counts_df['Name'] == level_name, 'Total'].values[0]
        cesev_weighted_sum += avg_score * count
        total_count += count
cesev_weighted_avg = cesev_weighted_sum / total_count

# Print the results
print(f"Weighted avg for NRELAP: {nrelap_weighted_avg:.2f}")
print(f"Weighted avg for CESEV: {cesev_weighted_avg:.2f}")

cat_normalized_brier= [nrelap_weighted_avg, cesev_weighted_avg]

Weighted avg for NRELAP: 1.07
Weighted avg for CESEV: 1.97


In [17]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Create a dictionary to store the scores for variables with the same letters before the '_'
variable_scores_dict = {}

# Iterate over each outcome variable in the folds
for level_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][level_name] 
        y_prob = concatenated_dfs[fold_index][level_name] 
        y_prob_avg = yi_test_dummies_avg[fold_index][level_name] 
        
        # Compute the Brier score and the normalized Brier score
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)
        normalized_brier = brier_score / brier_baseline

        # Append the normalized Brier score to the variable scores list
        variable_scores.append(normalized_brier)
    
    # Check if the variable name has letters before the '_'
    prefix = level_name.split('_')[0]
    
    # Add the normalized Brier scores to the dictionary based on the prefix
    if prefix in variable_scores_dict:
        variable_scores_dict[prefix].extend(variable_scores)
    else:
        variable_scores_dict[prefix] = variable_scores

# Compute the average and standard deviation of normalized Brier score for each prefix
for prefix, scores in variable_scores_dict.items():
    avg_score = np.mean(scores)
    std_score = np.std(scores)
    scores_with_std.append((prefix, avg_score, std_score))

#cat_normalized_brier = []
cat_std_brier = []
# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each categorical variable:")
for prefix, avg_score, std_score in scores_with_std:
    print(f"{prefix}: {avg_score:.2f} (± {std_score:.2f})")
    #cat_normalized_brier.append(avg_score)
    cat_std_brier.append(std_score)

Normalized Brier scores for each categorical variable:
NRELAP: 1.14 (± 0.10)
CESEV: 1.81 (± 0.35)


In [18]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]

In [19]:
# Initialize a list to store scores
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Check if the target variable is numerical or categorical
    if y_test_cv[0][variables.index(variable_name)].dtype.kind in 'bifc':
        # Compute scores for the variable across all folds
        for fold_index in range(len(y_test_cv)):
            y_test = y_test_cv[fold_index][variables.index(variable_name)] 
            y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
            
            score = normalized_mean_squared_error(y_test, y_pred)
            variable_scores.append(score)
        
        # Compute the average score for the variable across all folds
        variable_avg_score = np.mean(variable_scores)
        
        # Compute the standard deviation for the variable across all folds
        variable_std_score = np.std(variable_scores)
        
        # Append the tuple with three elements to the scores_with_std list
        scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

num_normalized_brier=[]
num_std_brier=[]
# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (local):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")
    num_normalized_brier.append(avg_score)
    num_std_brier.append(std_score)

Scores for each outcome (local):
KFSS_M-2y: 0.71 (± 0.05)
KFSS_P-2y: 0.85 (± 0.06)
EDSS-2y: 0.51 (± 0.04)
T25FW-2y: 1.11 (± 0.13)
NHPT-2y: 1.21 (± 0.22)
P_R36-SF12-after: 0.89 (± 0.05)
M_R36-SF12-after: 1.12 (± 0.06)
SES_after: 1.06 (± 0.06)
SLEC_after: 1.12 (± 0.06)
KFSS_M-after_2y: 0.79 (± 0.07)
KFSS_P-after_2y: 1.11 (± 0.17)
EDSS-after_2y: 0.63 (± 0.03)


In [20]:
combined_normalized_brier = np.concatenate((num_normalized_brier, cat_normalized_brier))
combined_normalized_brier

array([0.71423429, 0.85321741, 0.51328537, 1.1093239 , 1.20969817,
       0.89217309, 1.1248887 , 1.06490186, 1.11523591, 0.78858968,
       1.1066368 , 0.6293512 , 1.06717745, 1.97338779])

In [21]:
combined_std_brier = np.concatenate((num_std_brier, cat_std_brier))
combined_std_brier

array([0.05187301, 0.05755357, 0.04255014, 0.13374383, 0.22441086,
       0.04946904, 0.05924485, 0.05660492, 0.06158517, 0.06980832,
       0.16527459, 0.02855404, 0.09653535, 0.34554695])

In [22]:
# Compute the average
average_normalized_brier = np.mean(combined_normalized_brier)
print("Average normalized Brier score:", average_normalized_brier)

Average normalized Brier score: 1.0115786865178067


In [23]:
# Assuming you have an array of individual values to average
values_to_average = combined_normalized_brier  # Your individual values here

# Assuming you have an array of standard deviations corresponding to the individual values
individual_std_devs = combined_std_brier  # Your array of individual standard deviations here

# Step 1: Calculate the combined average
combined_average = np.mean(values_to_average)

# Step 2: Calculate the standard error of the mean (SEM)
sem = np.sqrt(np.sum(individual_std_devs**2) / len(values_to_average))

# Step 3: Calculate the standard deviation of the combined average
combined_std_dev = sem * np.sqrt(len(values_to_average))

# Now you have the standard deviation of the combined average
print("Standard deviation of the combined average:", combined_std_dev)

Standard deviation of the combined average: 0.5007408241700554


### Propagate predictions

In [24]:
y_pred_list = []
y_test_list = []
y_pred_prob_list = []
yi_test_dummies_list = []

for i in range(0, N_FOLDS): 
    Xi_train = features_cv[features_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = features_cv[features_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))

    # One hot encode categorical targets of test set to be able to compute brier score
    subset_yi_test = yi_test.select_dtypes(include=['object'])
    yi_test_dummies = pd.get_dummies(subset_yi_test, columns=subset_yi_test.columns, dtype=int)
    

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate="pred", #RUN MODELS IN A CHAIN
    )
    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_prob = chain.predict_proba(Xi_test)
    y_pred_list.append(y_pred)
    y_pred_prob_list.append(y_pred_prob)
    yi_test_dummies_list.append(yi_test_dummies)
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [25]:
yi_test_dummies_avg = []
# Calculate the percentage of 1s in each column
for yi_test_dummies_fold in yi_test_dummies_list:

    percentages = yi_test_dummies_fold.sum() / len(yi_test_dummies_fold)

    yi_test_dummies_avg_fold = pd.DataFrame(0, index=yi_test_dummies_fold.index, columns=yi_test_dummies_fold.columns)

    # Replace values in each column with the corresponding percentage
    for col in yi_test_dummies_avg_fold.columns:
        yi_test_dummies_avg_fold[col] = yi_test_dummies_fold[col].apply(lambda x: percentages[col])

    yi_test_dummies_avg.append(yi_test_dummies_avg_fold)

In [26]:
# Initialize an empty list to store the concatenated DataFrames
concatenated_dfs = []

# Iterate over each pair of arrays
for j, fold in enumerate(y_pred_prob_list):
    dfs = []
    len_array = 0
    
    for i, array in enumerate(fold):
        # Convert array to DataFrame
        col = yi_test_dummies_list[j].columns[len_array:len_array+len(array[0])]
        df = pd.DataFrame(array, columns=col, index=yi_test_dummies_list[j].index)
        dfs.append(df)
        len_array += len(array[0])
    
    # Concatenate DataFrames
    concatenated_df = pd.concat(dfs, axis=1)
    concatenated_dfs.append(concatenated_df)

# Now you should have a list of concatenated DataFrames

In [27]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Iterate over each outcome variable in the folds
for variable_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][variable_name] 
        y_prob = concatenated_dfs[fold_index][variable_name] 
        y_prob_avg = yi_test_dummies_avg[fold_index][variable_name] 
        
        # Check if the target variable is numerical or categorical
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)

        normalized_brier= brier_score/brier_baseline
                  
        variable_scores.append(normalized_brier)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each level:")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")

Normalized Brier scores for each level:
NRELAP_0.0: 0.98 (± 0.05)
NRELAP_1.0: 1.08 (± 0.03)
NRELAP_2.0: 1.10 (± 0.03)
NRELAP_3.0: 1.11 (± 0.11)
NRELAP_4+: 1.10 (± 0.08)
CESEV_MILD: 1.52 (± 0.17)
CESEV_MODERATE: 2.01 (± 0.17)
CESEV_SEVERE: 1.49 (± 0.13)


In [28]:
# Initialize dictionary to store column sums
column_sums = {}

# Iterate over each dataframe in reorganized_dummy_list_first
for df in yi_test_dummies_list:
    # Iterate over each column in the dataframe
    for column in df.columns:
        # Sum occurrences of 1s in the column and update column_sums
        column_sum = df[column].sum()
        column_sums[column] = column_sums.get(column, 0) + column_sum

# Create a dataframe from the column sums
total_counts_df = pd.DataFrame(list(column_sums.items()), columns=['Name', 'Total'])
total_counts_df

,Name,Total
0,NRELAP_0.0,1612
1,NRELAP_1.0,485
2,NRELAP_2.0,211
3,NRELAP_3.0,82
4,NRELAP_4+,75
5,CESEV_MILD,238
6,CESEV_MODERATE,534
7,CESEV_SEVERE,168


In [29]:
# Calculate the weighted sum for NRELAP
nrelap_weighted_sum = 0
total_count = 0
for level_name, avg_score, _ in scores_with_std:
    if level_name.startswith("NRELAP"):
        count = total_counts_df.loc[total_counts_df['Name'] == level_name, 'Total'].values[0]
        nrelap_weighted_sum += avg_score * count
        total_count += count
nrelap_weighted_avg = nrelap_weighted_sum / total_count

# Calculate the weighted sum for CESEV
cesev_weighted_sum = 0
total_count = 0
for level_name, avg_score, _ in scores_with_std:
    if level_name.startswith("CESEV"):
        count = total_counts_df.loc[total_counts_df['Name'] == level_name, 'Total'].values[0]
        cesev_weighted_sum += avg_score * count
        total_count += count
cesev_weighted_avg = cesev_weighted_sum / total_count

# Print the results
print(f"Weighted avg for NRELAP: {nrelap_weighted_avg:.2f}")
print(f"Weighted avg for CESEV: {cesev_weighted_avg:.2f}")

cat_normalized_brier = [nrelap_weighted_avg, cesev_weighted_avg]

Weighted avg for NRELAP: 1.02
Weighted avg for CESEV: 1.79


In [30]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Create a dictionary to store the scores for variables with the same letters before the '_'
variable_scores_dict = {}

# Iterate over each outcome variable in the folds
for level_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][level_name] 
        y_prob = concatenated_dfs[fold_index][level_name] 
        y_prob_avg = yi_test_dummies_avg[fold_index][level_name] 
        
        # Compute the Brier score and the normalized Brier score
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)
        normalized_brier = brier_score / brier_baseline

        # Append the normalized Brier score to the variable scores list
        variable_scores.append(normalized_brier)
    
    # Check if the variable name has letters before the '_'
    prefix = level_name.split('_')[0]
    
    # Add the normalized Brier scores to the dictionary based on the prefix
    if prefix in variable_scores_dict:
        variable_scores_dict[prefix].extend(variable_scores)
    else:
        variable_scores_dict[prefix] = variable_scores

# Compute the average and standard deviation of normalized Brier score for each prefix
for prefix, scores in variable_scores_dict.items():
    avg_score = np.mean(scores)
    std_score = np.std(scores)
    scores_with_std.append((prefix, avg_score, std_score))

#cat_normalized_brier = []
cat_std_brier = []
# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each categorical variable:")
for prefix, avg_score, std_score in scores_with_std:
    print(f"{prefix}: {avg_score:.2f} (± {std_score:.2f})")
    #cat_normalized_brier.append(avg_score)
    cat_std_brier.append(std_score)

Normalized Brier scores for each categorical variable:
NRELAP: 1.08 (± 0.08)
CESEV: 1.68 (± 0.28)


In [31]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]


# Initialize a list to store scores
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Check if the target variable is numerical or categorical
    if y_test_cv[0][variables.index(variable_name)].dtype.kind in 'bifc':
        # Compute scores for the variable across all folds
        for fold_index in range(len(y_test_cv)):
            y_test = y_test_cv[fold_index][variables.index(variable_name)] 
            y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
            
            score = normalized_mean_squared_error(y_test, y_pred)
            variable_scores.append(score)
        
        # Compute the average score for the variable across all folds
        variable_avg_score = np.mean(variable_scores)
        
        # Compute the standard deviation for the variable across all folds
        variable_std_score = np.std(variable_scores)
        
        # Append the tuple with three elements to the scores_with_std list
        scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

num_normalized_brier=[]
num_std_brier=[]
# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (chain - propagate predictions):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")
    num_normalized_brier.append(avg_score)
    num_std_brier.append(std_score)

Scores for each outcome (chain - propagate predictions):
KFSS_M-2y: 0.71 (± 0.05)
KFSS_P-2y: 0.84 (± 0.06)
EDSS-2y: 0.54 (± 0.05)
T25FW-2y: 1.13 (± 0.10)
NHPT-2y: 1.15 (± 0.24)
P_R36-SF12-after: 0.93 (± 0.04)
M_R36-SF12-after: 1.18 (± 0.05)
SES_after: 1.02 (± 0.06)
SLEC_after: 1.17 (± 0.07)
KFSS_M-after_2y: 0.83 (± 0.11)
KFSS_P-after_2y: 1.18 (± 0.18)
EDSS-after_2y: 0.71 (± 0.01)


In [32]:
combined_normalized_brier = np.concatenate((num_normalized_brier, cat_normalized_brier))
combined_normalized_brier

array([0.71423429, 0.83647109, 0.54140842, 1.1253104 , 1.1543414 ,
       0.92686274, 1.17520515, 1.02462275, 1.16888393, 0.82501172,
       1.18139744, 0.70775311, 1.01713966, 1.79418459])

In [33]:
combined_std_brier = np.concatenate((num_std_brier, cat_std_brier))
combined_std_brier

array([0.05187301, 0.06429934, 0.04790501, 0.09860199, 0.24117447,
       0.04417689, 0.05126601, 0.0609156 , 0.07305603, 0.11150221,
       0.18155296, 0.01340952, 0.0844205 , 0.28307411])

In [34]:
# Compute the average
average_normalized_brier = np.mean(combined_normalized_brier)
print("Average normalized Brier score:", average_normalized_brier)

Average normalized Brier score: 1.0137733349287035


In [35]:
# Assuming you have an array of individual values to average
values_to_average = combined_normalized_brier  # Your individual values here

# Assuming you have an array of standard deviations corresponding to the individual values
individual_std_devs = combined_std_brier  # Your array of individual standard deviations here

# Step 1: Calculate the combined average
combined_average = np.mean(values_to_average)

# Step 2: Calculate the standard error of the mean (SEM)
sem = np.sqrt(np.sum(individual_std_devs**2) / len(values_to_average))

# Step 3: Calculate the standard deviation of the combined average
combined_std_dev = sem * np.sqrt(len(values_to_average))

# Now you have the standard deviation of the combined average
print("Standard deviation of the combined average:", combined_std_dev)

Standard deviation of the combined average: 0.4727230495787199


### Propagate true values

In [36]:
y_pred_list = []
y_test_list = []
y_pred_prob_list = []
yi_test_dummies_list = []

for i in range(0, N_FOLDS): 
    Xi_train = features_cv[features_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    Xi_test = features_cv[features_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    yi_train = targets_cv[targets_cv['CV Fold'] != i].drop(["CV Fold"], axis=1)
    yi_test = targets_cv[targets_cv['CV Fold'] == i].drop(["CV Fold"], axis=1)
    y_test_list.append(pd.DataFrame(yi_test, columns=yi_test.columns, index=yi_test.index))

    # One hot encode categorical targets of test set to be able to compute brier score
    subset_yi_test = yi_test.select_dtypes(include=['object'])
    yi_test_dummies = pd.get_dummies(subset_yi_test, columns=subset_yi_test.columns, dtype=int)
    

    chain = Chain(
        model_reg=RandomForestRegressor(random_state=random_state),
        model_clf=RandomForestClassifier(random_state=random_state),
        propagate="true", #RUN MODELS IN A CHAIN
    )
    chain.fit(Xi_train, yi_train, target_types=None) #["reg","reg","reg","reg","reg","reg","reg","reg","reg","clf","clf"]
    y_pred = chain.predict(Xi_test)
    y_pred_prob = chain.predict_proba(Xi_test)
    y_pred_list.append(y_pred)
    y_pred_prob_list.append(y_pred_prob)
    yi_test_dummies_list.append(yi_test_dummies)
    print("Done with evaluating on CV Fold {}".format(i+1))

Done with evaluating on CV Fold 1
Done with evaluating on CV Fold 2
Done with evaluating on CV Fold 3
Done with evaluating on CV Fold 4
Done with evaluating on CV Fold 5


In [37]:
yi_test_dummies_list[0]

,NRELAP_0.0,NRELAP_1.0,NRELAP_2.0,NRELAP_3.0,NRELAP_4+,CESEV_MILD,CESEV_MODERATE,CESEV_SEVERE
19,0,1,0,0,0,0,1,0
24,1,0,0,0,0,0,0,0
25,0,1,0,0,0,0,0,1
34,0,0,1,0,0,0,0,1
39,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
2429,1,0,0,0,0,0,0,0
2437,0,0,0,1,0,0,1,0
2455,1,0,0,0,0,0,0,0
2460,1,0,0,0,0,0,0,0


In [38]:
yi_test_dummies_avg = []
# Calculate the percentage of 1s in each column
for yi_test_dummies_fold in yi_test_dummies_list:

    percentages = yi_test_dummies_fold.sum() / len(yi_test_dummies_fold)

    yi_test_dummies_avg_fold = pd.DataFrame(0, index=yi_test_dummies_fold.index, columns=yi_test_dummies_fold.columns)

    # Replace values in each column with the corresponding percentage
    for col in yi_test_dummies_avg_fold.columns:
        yi_test_dummies_avg_fold[col] = yi_test_dummies_fold[col].apply(lambda x: percentages[col])

    yi_test_dummies_avg.append(yi_test_dummies_avg_fold)

In [39]:
yi_test_dummies_avg[0]

,NRELAP_0.0,NRELAP_1.0,NRELAP_2.0,NRELAP_3.0,NRELAP_4+,CESEV_MILD,CESEV_MODERATE,CESEV_SEVERE
19,0.66,0.196,0.09,0.028,0.026,0.088,0.228,0.064
24,0.66,0.196,0.09,0.028,0.026,0.088,0.228,0.064
25,0.66,0.196,0.09,0.028,0.026,0.088,0.228,0.064
34,0.66,0.196,0.09,0.028,0.026,0.088,0.228,0.064
39,0.66,0.196,0.09,0.028,0.026,0.088,0.228,0.064
...,...,...,...,...,...,...,...,...
2429,0.66,0.196,0.09,0.028,0.026,0.088,0.228,0.064
2437,0.66,0.196,0.09,0.028,0.026,0.088,0.228,0.064
2455,0.66,0.196,0.09,0.028,0.026,0.088,0.228,0.064
2460,0.66,0.196,0.09,0.028,0.026,0.088,0.228,0.064


In [40]:
# Initialize an empty list to store the concatenated DataFrames
concatenated_dfs = []

# Iterate over each pair of arrays
for j, fold in enumerate(y_pred_prob_list):
    dfs = []
    len_array = 0
    
    for i, array in enumerate(fold):
        # Convert array to DataFrame
        col = yi_test_dummies_list[j].columns[len_array:len_array+len(array[0])]
        df = pd.DataFrame(array, columns=col, index=yi_test_dummies_list[j].index)
        dfs.append(df)
        len_array += len(array[0])
    
    # Concatenate DataFrames
    concatenated_df = pd.concat(dfs, axis=1)
    concatenated_dfs.append(concatenated_df)

# Now you should have a list of concatenated DataFrames

In [41]:
concatenated_dfs[4]

,NRELAP_0.0,NRELAP_1.0,NRELAP_2.0,NRELAP_3.0,NRELAP_4+,CESEV_MILD,CESEV_MODERATE,CESEV_SEVERE
2,0.79,0.08,0.11,0.01,0.01,0.10,0.760000,0.140000
4,0.43,0.18,0.18,0.06,0.15,0.06,0.880000,0.060000
5,0.39,0.24,0.26,0.04,0.07,0.13,0.760000,0.110000
10,0.38,0.26,0.24,0.09,0.03,0.10,0.770000,0.130000
13,0.64,0.16,0.16,0.03,0.01,0.14,0.700000,0.160000
...,...,...,...,...,...,...,...,...
2451,0.44,0.20,0.26,0.06,0.04,0.18,0.640000,0.180000
2453,0.49,0.31,0.17,0.02,0.01,0.20,0.620000,0.180000
2454,0.80,0.07,0.08,0.02,0.03,0.15,0.600000,0.250000
2458,0.68,0.11,0.17,0.03,0.01,0.16,0.630000,0.210000


In [42]:
yi_test_dummies_list[0]['NRELAP_0.0']

19      0
24      1
25      0
34      0
39      1
       ..
2429    1
2437    0
2455    1
2460    1
2464    0
Name: NRELAP_0.0, Length: 500, dtype: int32

In [43]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Iterate over each outcome variable in the folds
for variable_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][variable_name] 
        y_prob = concatenated_dfs[fold_index][variable_name] 
        y_prob_avg = yi_test_dummies_avg[fold_index][variable_name] 
        
        # Check if the target variable is numerical or categorical
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)

        normalized_brier= brier_score/brier_baseline
                  
        variable_scores.append(normalized_brier)
    
    # Compute the average score for the variable across all folds
    variable_avg_score = np.mean(variable_scores)
    
    # Compute the standard deviation for the variable across all folds
    variable_std_score = np.std(variable_scores)
    
    scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each level:")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")

Normalized Brier scores for each level:
NRELAP_0.0: 1.01 (± 0.02)
NRELAP_1.0: 1.01 (± 0.02)
NRELAP_2.0: 1.13 (± 0.10)
NRELAP_3.0: 1.02 (± 0.03)
NRELAP_4+: 1.07 (± 0.03)
CESEV_MILD: 1.18 (± 0.10)
CESEV_MODERATE: 2.26 (± 0.16)
CESEV_SEVERE: 1.22 (± 0.07)


In [44]:
# Initialize dictionary to store column sums
column_sums = {}

# Iterate over each dataframe in reorganized_dummy_list_first
for df in yi_test_dummies_list:
    # Iterate over each column in the dataframe
    for column in df.columns:
        # Sum occurrences of 1s in the column and update column_sums
        column_sum = df[column].sum()
        column_sums[column] = column_sums.get(column, 0) + column_sum

# Create a dataframe from the column sums
total_counts_df = pd.DataFrame(list(column_sums.items()), columns=['Name', 'Total'])
total_counts_df

,Name,Total
0,NRELAP_0.0,1612
1,NRELAP_1.0,485
2,NRELAP_2.0,211
3,NRELAP_3.0,82
4,NRELAP_4+,75
5,CESEV_MILD,238
6,CESEV_MODERATE,534
7,CESEV_SEVERE,168


In [45]:
# Calculate the weighted sum for NRELAP
nrelap_weighted_sum = 0
total_count = 0
for level_name, avg_score, _ in scores_with_std:
    if level_name.startswith("NRELAP"):
        count = total_counts_df.loc[total_counts_df['Name'] == level_name, 'Total'].values[0]
        nrelap_weighted_sum += avg_score * count
        total_count += count
nrelap_weighted_avg = nrelap_weighted_sum / total_count

# Calculate the weighted sum for CESEV
cesev_weighted_sum = 0
total_count = 0
for level_name, avg_score, _ in scores_with_std:
    if level_name.startswith("CESEV"):
        count = total_counts_df.loc[total_counts_df['Name'] == level_name, 'Total'].values[0]
        cesev_weighted_sum += avg_score * count
        total_count += count
cesev_weighted_avg = cesev_weighted_sum / total_count

# Print the results
print(f"Weighted avg for NRELAP: {nrelap_weighted_avg:.2f}")
print(f"Weighted avg for CESEV: {cesev_weighted_avg:.2f}")

cat_normalized_brier= [nrelap_weighted_avg, cesev_weighted_avg]

Weighted avg for NRELAP: 1.02
Weighted avg for CESEV: 1.80


In [46]:
# Initialize a list to store scores
scores_with_std = []
variables_cat = yi_test_dummies_list[0].columns

# Create a dictionary to store the scores for variables with the same letters before the '_'
variable_scores_dict = {}

# Iterate over each outcome variable in the folds
for level_name in variables_cat: 
    variable_scores = []
    
    # Compute scores for the variable across all folds
    for fold_index in range(len(yi_test_dummies_list)):
        y_test = yi_test_dummies_list[fold_index][level_name] 
        y_prob = concatenated_dfs[fold_index][level_name] 
        y_prob_avg = yi_test_dummies_avg[fold_index][level_name] 
        
        # Compute the Brier score and the normalized Brier score
        brier_score = brier_score_loss(y_test, y_prob)
        brier_baseline = brier_score_loss(y_test, y_prob_avg)
        normalized_brier = brier_score / brier_baseline

        # Append the normalized Brier score to the variable scores list
        variable_scores.append(normalized_brier)
    
    # Check if the variable name has letters before the '_'
    prefix = level_name.split('_')[0]
    
    # Add the normalized Brier scores to the dictionary based on the prefix
    if prefix in variable_scores_dict:
        variable_scores_dict[prefix].extend(variable_scores)
    else:
        variable_scores_dict[prefix] = variable_scores

# Compute the average and standard deviation of normalized Brier score for each prefix
for prefix, scores in variable_scores_dict.items():
    avg_score = np.mean(scores)
    std_score = np.std(scores)
    scores_with_std.append((prefix, avg_score, std_score))

#cat_normalized_brier = []
cat_std_brier = []
# Print the scores with average and standard deviation along with variable names
print("Normalized Brier scores for each categorical variable:")
for prefix, avg_score, std_score in scores_with_std:
    print(f"{prefix}: {avg_score:.2f} (± {std_score:.2f})")
    #cat_normalized_brier.append(avg_score)
    cat_std_brier.append(std_score)

Normalized Brier scores for each categorical variable:
NRELAP: 1.05 (± 0.07)
CESEV: 1.55 (± 0.51)


In [47]:
y_test_cv = []
y_pred_cv = []

for j in range(len(y_test_list)):  # 5
    y_test_targ = []
    y_pred_targ = []
    nvar=y_test_list[0].shape[1]

    for i in range(0, nvar):  # or (1, 5)
        missing_rows_mask = y_test_list[j].iloc[:, i].isna()
        y_test = y_test_list[j].iloc[:, i][~missing_rows_mask]
        y_pred = y_pred_list[j].iloc[:, i][~missing_rows_mask]
        
        y_test_targ.append(y_test)
        y_pred_targ.append(y_pred)
    
    y_test_cv.append(y_test_targ)
    y_pred_cv.append(y_pred_targ)
# y_test_cv[fold][outcome]

In [48]:
def normalized_mean_squared_error(true, pred):
    num = mse(true, pred)
    mean_value = np.mean(true)
    mean = np.full_like(true, mean_value)
    den = mse(true, mean)
    nmse_loss = num/den
    #rrmse_loss = np.sqrt(squared_error)
    return nmse_loss

In [49]:
# Initialize a list to store scores
scores_with_std = []

# Iterate over each outcome variable in the folds
for variable_name in variables: 
    variable_scores = []
    
    # Check if the target variable is numerical or categorical
    if y_test_cv[0][variables.index(variable_name)].dtype.kind in 'bifc':
        # Compute scores for the variable across all folds
        for fold_index in range(len(y_test_cv)):
            y_test = y_test_cv[fold_index][variables.index(variable_name)] 
            y_pred = y_pred_cv[fold_index][variables.index(variable_name)] 
            
            score = normalized_mean_squared_error(y_test, y_pred)
            variable_scores.append(score)
        
        # Compute the average score for the variable across all folds
        variable_avg_score = np.mean(variable_scores)
        
        # Compute the standard deviation for the variable across all folds
        variable_std_score = np.std(variable_scores)
        
        # Append the tuple with three elements to the scores_with_std list
        scores_with_std.append((variable_name, variable_avg_score, variable_std_score))

num_normalized_brier=[]
num_std_brier=[]
# Print the scores with average and standard deviation along with variable names
print("Scores for each outcome (chain - true values):")
for variable_name, avg_score, std_score in scores_with_std:
    print(f"{variable_name}: {avg_score:.2f} (± {std_score:.2f})")
    num_normalized_brier.append(avg_score)
    num_std_brier.append(std_score)

Scores for each outcome (chain - true values):
KFSS_M-2y: 0.71 (± 0.05)
KFSS_P-2y: 0.80 (± 0.03)
EDSS-2y: 0.48 (± 0.04)
T25FW-2y: 1.00 (± 0.03)
NHPT-2y: 0.94 (± 0.06)
P_R36-SF12-after: 0.81 (± 0.05)
M_R36-SF12-after: 1.02 (± 0.04)
SES_after: 0.98 (± 0.04)
SLEC_after: 1.05 (± 0.05)
KFSS_M-after_2y: 0.72 (± 0.11)
KFSS_P-after_2y: 1.03 (± 0.12)
EDSS-after_2y: 0.57 (± 0.04)


In [50]:
combined_normalized_brier = np.concatenate((num_normalized_brier, cat_normalized_brier))
combined_normalized_brier

array([0.71423429, 0.79807003, 0.48102205, 0.99509442, 0.93765468,
       0.81029509, 1.01698725, 0.98346546, 1.05271038, 0.72245778,
       1.03066155, 0.57338757, 1.02335419, 1.79777105])

In [51]:
combined_std_brier = np.concatenate((num_std_brier, cat_std_brier))
combined_std_brier

array([0.05187301, 0.03095861, 0.0357721 , 0.02596943, 0.06470454,
       0.04762939, 0.03626233, 0.03818469, 0.04752859, 0.11116757,
       0.12110202, 0.03834409, 0.06974685, 0.51230246])

In [52]:
# Compute the average
average_normalized_brier = np.mean(combined_normalized_brier)
print("Average normalized Brier score:", average_normalized_brier)

Average normalized Brier score: 0.9240832718716216


In [53]:
# Assuming you have an array of individual values to average
values_to_average = combined_normalized_brier  # Your individual values here

# Assuming you have an array of standard deviations corresponding to the individual values
individual_std_devs = combined_std_brier  # Your array of individual standard deviations here

# Step 1: Calculate the combined average
combined_average = np.mean(values_to_average)

# Step 2: Calculate the standard error of the mean (SEM)
sem = np.sqrt(np.sum(individual_std_devs**2) / len(values_to_average))

# Step 3: Calculate the standard deviation of the combined average
combined_std_dev = sem * np.sqrt(len(values_to_average))

# Now you have the standard deviation of the combined average
print("Standard deviation of the combined average:", combined_std_dev)

Standard deviation of the combined average: 0.5593774774318134
